In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import util
from scipy import sparse
import numpy as np

In [68]:
import sys
import codecs

def encode(direc):
    for datafile in os.listdir(direc):
        f = codecs.open(direc+'/'+datafile, 'r')
        contents = f.read()
        newcontents = contents.replace('&#','').replace(':\\', '').replace('\\', '').replace('.', '').replace('_', '').replace('-', '').replace(';', '')
        f.close()
        f = codecs.open(direc+'/'+datafile, 'w')
        f.write(newcontents)
        f.close()

In [69]:
encode('../train')

In [70]:
encode('../test')

In [71]:
direc = "../train"
classes = []
ids = [] 
train_files = []
for datafile in os.listdir(direc):
    train_files.append(direc+'/'+datafile)
    # extract id and true class (if available) from filename
    id_str,clazz = datafile.split('.')[:2]
    ids.append(id_str)
    # add target class if this is training data
    classes.append(util.malware_classes.index(clazz))

In [72]:
y_train = np.zeros((len(classes),len(util.malware_classes)))
y_train[np.arange(len(classes)), classes] = 1
y_train.shape

(3086, 15)

In [76]:
stop_words = []
for i in range (0,10000):
    stop_words.append(str(i))

In [77]:
TF = TfidfVectorizer('filename', analyzer = 'word', stop_words = stop_words, max_df = 0.4, min_df = 0.03)

In [78]:
X_train = TF.fit_transform(train_files)

In [79]:
X_train.shape

(3086, 2515)

In [7]:
sparse.save_npz("tfidf_train.npz", X_train)

In [8]:
# X = sparse.load_npz("tfidf_train.npz")
# X.shape

In [80]:
TF.vocabulary_

{u'local7a4ec452': 1539,
 u'programmatic': 1772,
 u'cdokumex7e1adminix7e1lokalex7e1temp': 489,
 u'hkeylocalmachinesoftwaremicrosoftctftipx7b1188450cfdab47ae80d8c9633f71be64x7dcategoryitemx7b5130a00955404fcf97ebaad33fc0ee09x7d': 1094,
 u'x24767a0000': 2384,
 u'explorermain': 868,
 u'clipbrdwndclass': 515,
 u'downloa': 744,
 u'6c3': 217,
 u'cwindowssystem32mlangdll': 624,
 u'ws232dllwsaaccept': 2090,
 u'msctfsharedmutexiog': 1629,
 u'travelhawaii': 1970,
 u'dupeexe': 760,
 u'createthreadremote': 584,
 u'usetimermethod': 2031,
 u'x24': 2128,
 u'disableunccheck': 734,
 u'itemspenis': 1269,
 u'entertainmentdating': 818,
 u'6622017154': 205,
 u'hkeylocalmachinesoftwaremicrosoftwindowscurrentversionpoliciesexplorer': 1125,
 u'336ca6489': 137,
 u'hkeylocalmachinesoftwareclassestypelibx7b0002043000000000c000000000000046x7d200win32': 1078,
 u'x2471a16a55': 2310,
 u'x2447161000': 2264,
 u'02x26ticksboot': 14,
 u'57c470b423ac9d8db9ad8e3a28c45cf8f86f329e': 184,
 u'cwindowssystem32msvbvm60dll': 635,

In [10]:
TF.stop_words_

{u'9l2u',
 u'w0lx',
 u'js890673',
 u'1ec0e280caba',
 u'35367caac37ec52f48f8628762293ee07958dad1',
 u'__vbafreeobj',
 u'j5py',
 u'b3efbe',
 u'56cc0a27',
 u'8mm6',
 u'035e8a5b',
 u'gaf',
 u'7hqizk',
 u'uwvs',
 u'c2fb088f1802',
 u'5988',
 u'5989',
 u'db26f49f',
 u'aaaaaaaaaaaaaain',
 u'4cb48ebb4a74',
 u'5980',
 u'5981',
 u'5986',
 u'5987',
 u'e102fe3a01ca1c2d000004c81',
 u'5985',
 u'xodf',
 u'glqvcr',
 u'js8906730',
 u'002500f2',
 u'8bbefd1d4b25',
 u'abcaf513bcb6',
 u'8mmg',
 u'sowell',
 u'950fd59236e',
 u'37461f682a983e',
 u'cf7252f08718a7c3b7e2012183222db6fdae83ac',
 u'rassapi',
 u'qujdqjperuxbrkzjrdowmdazmdeymdpdolxet0tvtuv_mvxbre1jtkl_mvxbtldftkr_mvxqt0tfu0v_mvxpuevou0t_ms5fwev8c2vhcnnomjawlnnvbxwwmtewmdawmq',
 u'wood2',
 u'wood3',
 u'jrpn',
 u'29734814512080',
 u'43bwagn3gsru1uim',
 u'f3aedc804044',
 u'berkshir',
 u'5eddc172',
 u'wooden',
 u'abff33a7256843ed51cd359a9f5521dbb2a25f2f',
 u'pcoptimizer',
 u'hsu',
 u'pdhhj',
 u'okayrd',
 u'001a0138',
 u'd008d1a6dc5ca1',
 u'598c',
 u'598a'

In [81]:
len(TF.stop_words_)

888088

In [82]:
test_direc = "../test"
test_files = []
test_ids = []
for datafile in os.listdir(test_direc):
    test_files.append(test_direc+'/'+datafile)
    id_str,clazz = datafile.split('.')[:2]
    test_ids.append(id_str)

In [83]:
X_test = TF.transform(test_files)
X_test.shape

(3724, 2515)

In [84]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators = 100, n_jobs = -1)
RF.fit(X_train, y_train)
print RF.n_features_
print RF.feature_importances_
print RF.score(X_train, y_train)

2515
[1.83097031e-03 1.69030056e-03 9.42450989e-04 ... 2.39709752e-03
 3.91736118e-05 0.00000000e+00]
0.9896305897602073


In [91]:
t = 0
for i in RF.feature_importances_:
    if i > 0.001:
        print i
        t += 1
print t

0.0018309703145859754
0.0016903005647884509
0.0010158279037817974
0.0036184466861231873
0.002580790145427247
0.007364046668190287
0.005567863404105625
0.004554399311029112
0.0019015023721359555
0.0021296095790762624
0.009694228796187608
0.0012673708364636241
0.004908998398851237
0.001686459128164085
0.004511492790803164
0.0010763126051742203
0.0010995076121611633
0.0025954446625415157
0.0025920759363027386
0.005149543092688283
0.0017343896299899853
0.0051146378419266005
0.0015019200929755607
0.003953140270518204
0.001094410371096222
0.001612262464711359
0.001727616450877734
0.0016249295415901675
0.004965451620686664
0.002159557799331461
0.001920320202603075
0.0015592902322152308
0.007027798341961187
0.0010434221676058606
0.002595295922769628
0.0012254136008878016
0.004109974617722597
0.004049028444677013
0.010428211518384023
0.0037582136232544856
0.0024517541625054392
0.0022987442424030795
0.00111210714438913
0.005198383311715592
0.0019268749978233893
0.004548602325847357
0.00273148527

In [92]:
print RF.feature_importances_[745]

8.450881217686475e-06


In [124]:
from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier(loss = 'modified_huber', n_jobs = -1)
SGD.fit(X_train, classes)
print SGD.score(X_train, classes)

0.9141283214517174


In [125]:
print "making predictions..."
#preds = RF.predict(X_test)
#results = np.argmax(preds, axis=1)

results = SGD.predict(X_test)

making predictions...


In [126]:
print "writing predictions..."
outputfile = "tfidf_predictions.csv"
util.write_predictions(results, test_ids, outputfile)

writing predictions...


In [127]:
import csv 
import sys 

def reorder_submission(file_to_reorder, newfile_name = "experiment_results.csv"):
    # READ IN KEYS IN CORRECT ORDER AS LIST
    with open('keys.csv','r') as f:
        keyreader = csv.reader(f)
        keys = [key[0] for key in keyreader]

    # READ IN ALL PREDICTIONS, REGARDLESS OF ORDER
    with open(file_to_reorder) as f:
        oldfile_reader = csv.reader(f)
        D = {}
        for i,row in enumerate(oldfile_reader):
            if i == 0:
                continue
            _id, pred = row 
            D[_id] = pred

    # WRITE PREDICTIONS IN NEW ORDER
    with open(newfile_name,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('Id','Prediction'))
        for key in keys:
            writer.writerow((key,D[key]))

    print("".join(["Reordered ", file_to_reorder," and wrote to ", newfile_name]))

In [128]:
reorder_submission(outputfile, "tfidf_sgd_results.csv")

Reordered tfidf_predictions.csv and wrote to tfidf_sgd_results.csv
